<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=990bb291d1d4cb0553df1611c25887c209323b046a74eeb3f1f474f74a09dbe5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.3
    Uninstalling protobuf-6.33.3:
      Successfully uninstalled protobuf-6.33.3


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-12 10:29:59
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -68.76 K (-0.49%)
Current PnL: -28.72 L (-18.32%)
CY Booked + Current PnL: -14.43 L (-9.2%)
-------------------
Total profit:  1.68 L
Total loss:  -30.41 L
-------------------
Total Booked + Current PnL: 12.55 L (9.66%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.18 L (67.1%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.22%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.25,14.25,6.26,21.40,18983.0,37815.0,303241.0,5.95,58.0,X-LC,5.83,14.0,1.99,2.16,21.48,XY25,NTT,FMCG
17,COALINDIA,484.83,2.81,5.77,12.72,19.23,20242.0,8676.0,159137.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.27,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-1.43,-21.84,28.06,0.09,22103.0,-22007.0,78770.0,82.92,35.0,M-SC,3.32,253.0,-1.00,0.56,3.49,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.71,-4.23,27.43,22.05,25740.0,-4140.0,93840.0,-17.34,46.0,H-SC,6.27,168.0,-0.16,0.67,36.40,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.02,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.31,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.81,5.77,12.72,19.23,20242.0,8676.0,159137.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.27,XY25,ATH,MINING
18,COFFEEDAY,80.00,2.50,-41.71,132.09,35.27,87420.0,-47367.0,66182.0,-55.73,45.0,L-SC,4.43,268.0,-0.54,0.47,61.98,XR,NTT,HOTELS
37,IEX,219.00,2.17,-4.06,54.92,48.63,105351.0,-8128.0,191826.0,-36.24,51.0,H-SC,13.77,137.0,-0.08,1.37,7.08,XR,NTT,MISC
57,RECLTD,446.00,1.38,-0.22,21.00,20.74,42573.0,-440.0,202730.0,31.15,55.0,M-MC,5.47,189.0,-0.01,1.44,10.38,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,1.02,-34.20,101.96,32.89,51797.0,-26406.0,50801.0,-727.32,60.0,L-MC,7.83,259.0,-0.51,0.36,39.90,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,518.0,-3.37,-68.51,217.79,0.07,94074.0,-93982.0,43195.0,1483.33,32.0,L-SC,2.07,269.0,-1.00,0.31,7.43,OX40N,NTT,JEWELLERY
48,LAOPALA,464.0,-2.60,-41.79,138.94,39.09,106035.0,-54788.0,76317.0,66.91,26.0,H-SC,13.80,155.0,-0.52,0.54,0.00,AR,NTT,CERAMICS
76,TRIDENT,48.0,-2.52,-31.65,88.01,28.51,55498.0,-29196.0,63059.0,-19.07,33.0,M-SC,7.75,226.0,-0.53,0.45,7.90,XR,NTT,TEXTILES
21,DEN,75.0,-2.33,-42.75,148.18,42.07,71648.0,-36112.0,48352.0,70.98,37.0,M-SC,7.76,236.0,-0.50,0.34,2.68,AR,NTT,ENTERTAINMENT
65,SIEMENS,4671.5,-2.13,-19.96,56.81,25.51,84621.0,-37140.0,148955.0,-8.57,36.0,H-LC,1.95,50.0,-0.44,1.06,9.77,AR,ATH,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,1.38,-0.22,21.0,20.74,42573.0,-440.0,202730.0,31.15,55.0,M-MC,5.47,189.0,-0.01,1.44,10.38,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.49,-3.84,116.22,107.91,164998.0,-5674.0,141970.0,-21.77,46.0,M-SC,10.10,220.0,-0.03,1.01,5.86,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.02,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.31,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.82,-18.65,62.30,32.03,66814.0,-24589.0,107246.0,-12.01,62.0,H-SC,6.83,157.0,-0.37,0.76,20.12,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-1.43,-21.84,28.06,0.09,22103.0,-22007.0,78770.0,82.92,35.0,M-SC,3.32,253.0,-1.00,0.56,3.49,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.44,-22.76,46.91,13.47,97707.0,-61380.0,208287.0,-68.31,46.0,H-SC,4.50,159.0,-0.63,1.48,5.91,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.81,5.77,12.72,19.23,20242.0,8676.0,159137.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.27,XY25,ATH,MINING
84,WIPRO,420.00,0.23,7.69,59.97,72.27,109428.0,13024.0,182472.0,-7.44,50.0,M-LC,6.18,101.0,0.12,1.30,14.34,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.23,7.69,59.97,72.27,109428.0,13024.0,182472.0,-7.44,50.0,M-LC,6.18,101.0,0.12,1.30,14.34,XR,NTT,IT
25,FINCABLES,1641.55,-1.49,-3.84,116.22,107.91,164998.0,-5674.0,141970.0,-21.77,46.0,M-SC,10.10,220.0,-0.03,1.01,5.86,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.95,-8.68,126.38,106.72,142335.0,-10711.0,112625.0,-54.82,32.0,H-SC,7.69,138.0,-0.08,0.80,12.66,AR,ATH,MISC
37,IEX,219.00,2.17,-4.06,54.92,48.63,105351.0,-8128.0,191826.0,-36.24,51.0,H-SC,13.77,137.0,-0.08,1.37,7.08,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.77,-6.36,44.84,35.63,87998.0,-13326.0,196248.0,-19.40,33.0,H-MC,3.37,119.0,-0.15,1.40,9.25,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.25,-17.16,33.73,10.78,67150.0,-41236.0,199082.0,-50.55,13.0,X-LC,7.09,1.0,-0.61,1.42,0.25,X40,NTT,FMCG
7,AWL,485.00,-1.56,-28.86,120.96,57.19,280642.0,-94115.0,232012.0,-66.31,21.0,X-MC,9.84,58.0,-0.34,1.65,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.64,-41.24,130.09,35.20,98360.0,-53061.0,75609.0,-0.94,27.0,X-SC,17.47,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
52,PAGEIND,51605.08,0.64,-12.68,49.58,30.62,102631.0,-30054.0,207000.0,-37.95,29.0,X-MC,16.64,57.0,-0.29,1.48,0.64,X40,ATH,APPARELS
78,UNITDSPR,1644.00,-0.83,-3.04,24.55,20.77,55734.0,-7121.0,227023.0,-11.53,29.0,X-MC,1.33,70.0,-0.13,1.62,3.67,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.83,-3.04,24.55,20.77,55734.0,-7121.0,227023.0,-11.53,29.0,X-MC,1.33,70.0,-0.13,1.62,3.67,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-0.47,-8.04,35.36,24.48,86941.0,-21491.0,245874.0,-9.62,37.0,X-LC,1.55,37.0,-0.25,1.75,42.52,XY24,BTT,FINANCE
65,SIEMENS,4671.50,-2.13,-19.96,56.81,25.51,84621.0,-37140.0,148955.0,-8.57,36.0,H-LC,1.95,50.0,-0.44,1.06,9.77,AR,ATH,ELECTRICAL
81,VBL,671.64,0.03,-1.19,37.31,35.67,116436.0,-3764.0,312078.0,-12.96,53.0,X-LC,2.26,5.0,-0.03,2.22,12.50,X40N,ATH,FMCG
73,TCS,4389.97,-0.40,-12.15,37.41,20.71,124302.0,-45967.0,332270.0,-24.27,46.0,X-LC,2.32,3.0,-0.37,2.37,11.02,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.09,-28.49,129.55,64.15,220442.0,-67791.0,170160.0,-58.06,32.0,X-SC,6.39,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-1.56,-28.86,120.96,57.19,280642.0,-94115.0,232012.0,-66.31,21.0,X-MC,9.84,58.0,-0.34,1.65,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.64,-41.24,130.09,35.20,98360.0,-53061.0,75609.0,-0.94,27.0,X-SC,17.47,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.65,-6.73,43.87,34.19,87368.0,-14368.0,199152.0,-34.39,33.0,X-MC,4.07,60.0,-0.16,1.42,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-1.62,-49.00,198.63,52.31,147054.0,-71126.0,74034.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.99,-28.90,100.92,42.84,46641.0,-18790.0,46216.0,-53.72,47.0,X-SC,26.53,83.0,-0.40,0.33,4.56,XY24,NTT,MISC
58,RELAXO,1176.00,-1.62,-49.00,198.63,52.31,147054.0,-71126.0,74034.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.64,-41.24,130.09,35.20,98360.0,-53061.0,75609.0,-0.94,27.0,X-SC,17.47,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.10,-1.96,26.55,24.08,33045.0,-2485.0,124465.0,-9.82,46.0,X-SC,6.21,91.0,-0.08,0.89,17.27,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.69,-18.29,75.41,43.32,100356.0,-29792.0,133080.0,-29.79,40.0,X-SC,7.95,90.0,-0.30,0.95,2.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.25,-17.16,33.73,10.78,67150.0,-41236.0,199082.0,-50.55,13.0,X-LC,7.09,1.0,-0.61,1.42,0.25,X40,NTT,FMCG
75,TMPV,600.00,-0.52,-27.79,70.31,22.98,114686.0,-62783.0,163115.0,-25.52,40.0,X-LC,4.08,2.0,-0.55,1.16,2.59,XY24,NTT,AUTO
73,TCS,4389.97,-0.40,-12.15,37.41,20.71,124302.0,-45967.0,332270.0,-24.27,46.0,X-LC,2.32,3.0,-0.37,2.37,11.02,X40,ATH,IT
81,VBL,671.64,0.03,-1.19,37.31,35.67,116436.0,-3764.0,312078.0,-12.96,53.0,X-LC,2.26,5.0,-0.03,2.22,12.50,X40N,ATH,FMCG
41,INFY,1972.00,-1.00,8.78,23.40,34.24,79648.0,27486.0,340374.0,-15.29,46.0,X-LC,7.64,6.0,0.35,2.43,17.79,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.02,-34.20,101.96,32.89,51797.0,-26406.0,50801.0,-727.32,60.0,L-MC,7.83,259.0,-0.51,0.36,39.90,XR,NTT,BANKS
13,BSOFT,831.70,-1.65,-24.74,100.36,50.79,106231.0,-34803.0,105850.0,-3.36,44.0,H-SC,10.36,171.0,-0.33,0.75,23.24,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.02,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.31,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS
17,COALINDIA,484.83,2.81,5.77,12.72,19.23,20242.0,8676.0,159137.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.27,XY25,ATH,MINING
84,WIPRO,420.00,0.23,7.69,59.97,72.27,109428.0,13024.0,182472.0,-7.44,50.0,M-LC,6.18,101.0,0.12,1.30,14.34,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.36,-31.84,65.64,12.90,97756.0,-69568.0,148928.0,119.95,56.0,H-SC,10.05,173.0,-0.71,1.06,23.83,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.82,-18.65,62.30,32.03,66814.0,-24589.0,107246.0,-12.01,62.0,H-SC,6.83,157.0,-0.37,0.76,20.12,OX40N,NTT,IT
17,COALINDIA,484.83,2.81,5.77,12.72,19.23,20242.0,8676.0,159137.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.27,XY25,ATH,MINING
57,RECLTD,446.00,1.38,-0.22,21.00,20.74,42573.0,-440.0,202730.0,31.15,55.0,M-MC,5.47,189.0,-0.01,1.44,10.38,XY25,NTT,FINANCE
82,VOLTAS,1856.00,0.45,15.26,25.97,45.20,57396.0,29268.0,221010.0,4.03,63.0,X-MC,5.22,80.0,0.51,1.57,23.10,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.52
2,50,76.61


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.57
MC    30.82
LC    26.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.37
X40      24.01
X40N     12.80
XY25     10.81
XR        8.96
AR        8.50
OX40N     7.60
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.42
X-LC    23.12
H-SC    22.78
M-SC    11.21
X-SC     7.80
H-MC     4.60
M-MC     1.44
M-LC     1.30
L-LC     1.13
H-LC     1.06
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.13,-9.45,45.37
IT,12.88,-18.32,78.00
FINANCE,11.11,-14.32,62.31
MISC,7.06,-31.15,84.80
ELECTRICAL,5.90,-13.28,54.77
PAINTS,5.81,-13.30,30.13
INSURANCE,4.76,-1.63,36.41
PHARMA,3.94,-6.35,39.82
AUTO,2.78,-34.49,81.12


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3271154.0,21
AR,1378765.0,10
XR,1233996.0,12
X40,1153855.0,15
X40N,946145.0,9
OX40N,724219.0,10
XY25,422726.0,7
SR,287036.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3596174.0,24
X-MC,1605579.0,16
M-SC,1486602.0,15
X-LC,980926.0,13
X-SC,824738.0,8
H-MC,433722.0,3
L-SC,181494.0,2
M-LC,109428.0,1
H-LC,84621.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260694.0      6
           AR         933347.0      5
M-SC       XY24       852911.0      6
H-SC       XR         797815.0      7
X-MC       X40        521880.0      7
           XY24       449577.0      3
X-LC       X40        430775.0      6
X-MC       X40N       410894.0      4
X-SC       X40N       323410.0      3
H-SC       OX40N      317282.0      4
M-SC       OX40N      312863.0      5
X-SC       XY24       300128.0      3
H-SC       SR         287036.0      2
H-MC       AR         227505.0      2
X-MC       XY25       223228.0      2
X-LC       X40N       211841.0      2
H-MC       XY24       206217.0      1
X-LC       XY24       201627.0      2
X-SC       X40        201200.0      2
M-SC       XR         187536.0      2
X-LC       XY25       136683.0      3
M-SC       AR         133292.0      2
M-LC       XR         109428.0      1
L-SC       OX40N       94074.0      1
           XR          87420.0      1
H-LC       AR          84621.0      1
L-MC       XR          51797.0      1
M-MC       XY25        42573.0      1
L-LC       XY25        20242.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
